In [48]:
import random
import simpy
from collections import namedtuple
import pandas as pd

In [38]:
# system attributes
RANDOM_SEED = 42
NUM_REGISTERS = 2  # NUMBER OF REGISTERS AT THE COUNTER
CHECKOUTTIME = 5      # MINUTES IT TAKES TO CHECK OUT
T_INTER = 3       # CUSTOMER INTERARRIVAL TIME
SIM_TIME = 20  # SIMULATION RUN TIME

In [39]:
class Counter(object):
    def __init__(self, env, num_registers, checkout_time):
        self.env = env
        self.register = simpy.Resource(env, num_registers)
        self.checkout_time = checkout_time

    def check_out(self, customer):
        yield self.env.timeout(CHECKOUTTIME)


In [40]:
def customer(env, name, counter, wait_times):
    print(f"Customer {name} enters the checkout line at time {env.now}")
    arrival_time = env.now
    with counter.register.request() as request:
        yield request

        print(f"Customer {name} begins checking out at time {env.now}")
        checkout_start_time = env.now
        yield env.process(counter.check_out(name))

        print(f"Customer {name} is done checking out at time {env.now}")
        wait_times.append(cust(name, checkout_start_time - arrival_time))


In [41]:
def setup(env, num_registers, checkout_time, t_inter, wait_times):

    counter = Counter(env, num_registers, checkout_time)

    # Create 2 initial customers
    for i in range(2):
        env.process(customer(env, i, counter, wait_times))

    # Continuously create customers
    while True:
        yield env.timeout(random.randint(t_inter - 2, t_inter + 2))
        i += 1
        env.process(customer(env, i, counter, wait_times))


In [45]:
# monitoring variables
cust = namedtuple("cust", "name, wait_time")
WAIT_TIMES = []

In [49]:
def run():
    random.seed(RANDOM_SEED)
    
    # creat a simpy environment and process the setup function
    env = simpy.Environment()
    env.process(setup(env, NUM_REGISTERS, CHECKOUTTIME, T_INTER, WAIT_TIMES))

    # execute the sumlation!
    env.run(until=SIM_TIME)

Checkout
Customer 0 enters the checkout line at time 0
Customer 1 enters the checkout line at time 0
Customer 0 begins checking out at time 0
Customer 1 begins checking out at time 0
Customer 2 enters the checkout line at time 1
Customer 3 enters the checkout line at time 2
Customer 4 enters the checkout line at time 5
Customer 0 is done checking out at time 5
Customer 1 is done checking out at time 5
Customer 2 begins checking out at time 5
Customer 3 begins checking out at time 5
Customer 5 enters the checkout line at time 7
Customer 6 enters the checkout line at time 9
Customer 2 is done checking out at time 10
Customer 3 is done checking out at time 10
Customer 4 begins checking out at time 10
Customer 5 begins checking out at time 10
Customer 7 enters the checkout line at time 11
Customer 8 enters the checkout line at time 12
Customer 4 is done checking out at time 15
Customer 5 is done checking out at time 15
Customer 6 begins checking out at time 15
Customer 7 begins checking ou

In [52]:
sim_results = pd.DataFrame(WAIT_TIMES)
sim_results

,name,wait_time
0,0,0
1,1,0
2,2,4
3,3,3
4,4,5
5,5,3
6,0,0
7,1,0
8,2,4
9,3,3


In [53]:
sim_results.wait_time.mean()

2.5